In [1]:
import urllib.request
urllib.request.urlretrieve ("https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz", "a.tar.gz")


('a.tar.gz', <http.client.HTTPMessage at 0x7f581a240518>)

In [0]:
import tarfile
tar = tarfile.open("a.tar.gz")
tar.extractall()
tar.close()

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pickle


def unpickle(file):
 '''Load byte data from file'''
 with open(file, 'rb') as f:
  data = pickle.load(f, encoding='latin-1')
  return data


def load_cifar10_data(data_dir):
 '''Return train_data, train_labels, test_data, test_labels
 The shape of data is 32 x 32 x3'''
 train_data = None
 train_labels = []

 for i in range(1, 6):
  data_dic = unpickle(data_dir + "/data_batch_{}".format(i))
  if i == 1:
   train_data = data_dic['data']
  else:
   train_data = np.vstack((train_data, data_dic['data']))
  train_labels += data_dic['labels']

 test_data_dic = unpickle(data_dir + "/test_batch")
 test_data = test_data_dic['data']
 test_labels = test_data_dic['labels']

 train_data = train_data.reshape((len(train_data), 3, 32, 32))
 train_data = np.rollaxis(train_data, 1, 4)
 train_labels = np.array(train_labels)

 test_data = test_data.reshape((len(test_data), 3, 32, 32))
 test_data = np.rollaxis(test_data, 1, 4)
 test_labels = np.array(test_labels)

 return train_data, train_labels, test_data, test_labels

data_dir = 'cifar-10-batches-py'

train_data, train_labels, test_data, test_labels = load_cifar10_data(data_dir)

print(train_data.shape)
print(train_labels.shape)

print(test_data.shape)
print(test_labels.shape)


(50000, 32, 32, 3)
(50000,)
(10000, 32, 32, 3)
(10000,)


In [0]:
#importing required libraries

#image precessing
import cv2
#file handling
import os
#obvious
import numpy as np
import pandas as pd
#library to know the progress of or loops
from tqdm import tqdm

In [5]:
name,count = np.unique(test_labels,return_counts=True)
print(count)

[1000 1000 1000 1000 1000 1000 1000 1000 1000 1000]


In [0]:
x_train = train_data
x_test = test_data

In [7]:
train_labels


array([6, 9, 9, ..., 9, 1, 1])

In [0]:
#creating y
def func(a):
  y = []
  for name in a:
    if name == 0:
      y.append((1,0,0,0,0,0,0,0,0,0))
    elif name == 1:
      y.append((0,1,0,0,0,0,0,0,0,0))
    elif name == 2:
      y.append((0,0,1,0,0,0,0,0,0,0))
    elif name == 3:
      y.append((0,0,0,1,0,0,0,0,0,0))
    elif name == 4:
      y.append((0,0,0,0,1,0,0,0,0,0))
    elif name == 5:
      y.append((0,0,0,0,0,1,0,0,0,0))
    elif name == 6:
      y.append((0,0,0,0,0,0,1,0,0,0))
    elif name == 7:
      y.append((0,0,0,0,0,0,0,1,0,0))
    elif name == 8:
      y.append((0,0,0,0,0,0,0,0,1,0))
    elif name == 9:
      y.append((0,0,0,0,0,0,0,0,0,1))
  return np.array(y)


In [0]:
y_train = func(train_labels)
y_test = func(test_labels)

In [10]:
w = y_train
type(w)

numpy.ndarray

In [0]:
x_train = np.reshape(x_train,(50000,3072))
x_test = np.reshape(x_test,(10000,3072))

In [0]:
x_train = x_train / 255
x_test = x_test / 255

In [0]:
import tensorflow as tf

In [0]:
input_width = 32
input_height = 32
input_channels = 3
input_pixels = 3072

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1
conv1_k = 5
conv2_k = 5
max_pool1_k = 2
max_pool2_k = 2

n_hidden = 1024
n_out = 10

input_size_to_hidden = (input_width//(max_pool1_k*max_pool2_k)) * (input_height//(max_pool1_k*max_pool2_k)) *n_conv2
n_batch = 100
buffer_size = 1024

In [15]:
# create input pipeline using tf.data
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
dataset = dataset.shuffle(buffer_size=buffer_size)
dataset = dataset.apply(tf.contrib.data.batch_and_drop_remainder(batch_size=n_batch))
iterator = dataset.make_initializable_iterator()

Instructions for updating:
Use `tf.data.Dataset.batch(..., drop_remainder=True)`.


In [0]:
# create batch iterators
input_mini_batch, label_mini_batch = iterator.get_next()

In [0]:
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.shuffle(buffer_size=buffer_size)
dataset = dataset.apply(tf.contrib.data.batch_and_drop_remainder(batch_size=n_batch))
test_iterator = test_dataset.make_initializable_iterator()

In [0]:
# create batch iterators
X_test_mini_batch, y_test_mini_batch = test_iterator.get_next()

In [0]:
weights = {
    "wc1" : tf.Variable(tf.random_normal([conv1_k, conv1_k, input_channels, n_conv1])),
    "wc2" : tf.Variable(tf.random_normal([conv2_k, conv2_k, n_conv1, n_conv2])),
    "wh1" : tf.Variable(tf.random_normal([input_size_to_hidden, n_hidden])),
    "wo" : tf.Variable(tf.random_normal([n_hidden, n_out]))
}

biases = {
    "bc1" : tf.Variable(tf.random_normal([n_conv1])),
    "bc2" : tf.Variable(tf.random_normal([n_conv2])),
    "bh1" : tf.Variable(tf.random_normal([n_hidden])),
    "bo" : tf.Variable(tf.random_normal([n_out])),
}

In [0]:
def conv(x, weights, bias, strides = 1):
    out = tf.nn.conv2d(x, weights, padding="SAME", strides = [1, strides, strides, 1])
    out = tf.nn.bias_add(out, bias)
    out = tf.nn.relu(out)
    return out

def maxpooling(x, k = 2):
    return tf.nn.max_pool(x, padding = "SAME", ksize = [1, k, k, 1], strides = [1, k, k, 1])

In [0]:
def cnn(x, weights, biases, keep_prob):
    x = tf.reshape(x, shape = [-1 ,input_height, input_width, input_channels])
    conv1 = conv(x, weights['wc1'], biases['bc1'], stride_conv1)
    conv1_pool = maxpooling(conv1, max_pool1_k)
    
    conv2 = conv(conv1_pool, weights['wc2'], biases['bc2'], stride_conv2)
    conv2_pool = maxpooling(conv2, max_pool2_k)
    
    hidden_input = tf.reshape(conv2_pool, shape = [-1, input_size_to_hidden])
    hidden_output_before_activation = tf.add(tf.matmul(hidden_input, weights['wh1']), biases['bh1'])
    hidden_output_before_dropout = tf.nn.relu(hidden_output_before_activation)
    hidden_output = tf.nn.dropout(hidden_output_before_dropout, keep_prob) 
   
    output = tf.add(tf.matmul(hidden_output, weights['wo']), biases['bo'])
    return output

In [0]:
x = tf.placeholder("float", [None, input_pixels])
y = tf.placeholder(tf.int32, [None, n_out])
keep_prob = tf.placeholder("float")
pred = cnn(x, weights, biases, keep_prob)

In [0]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=pred, labels = y))

In [0]:
optimizer = tf.train.AdamOptimizer()
optimize = optimizer.minimize(cost)

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [0]:
from tqdm import tqdm

In [0]:
def func2(x1,y1):
  predictions = tf.argmax(pred, 1)
  correct_labels = tf.argmax(y, 1)
  correct_predictions = tf.equal(predictions, correct_labels)
  predictions,correct_preds  = sess.run([predictions, correct_predictions], feed_dict={x:x1,
                                              y:y1, keep_prob:1.0})
  return correct_preds.sum()

In [30]:
n_epochs = 50
for epoch in range(0,n_epochs):
    epoch_loss = 0
    sess.run(iterator.initializer)
    
    while True:
        try:
            X_for_train, y_for_train = sess.run([input_mini_batch, label_mini_batch])
            t, c = sess.run([optimize, cost], feed_dict={x: X_for_train, y: y_for_train, keep_prob: 1.0})
            epoch_loss += c
        except tf.errors.OutOfRangeError:
            break

    # print loss
    print(f"Epoch {epoch} out of {n_epochs}, loss: {epoch_loss}")
    
    # print train and test accuracies
    # calculate_accuracy()
    print("Accuracy: ", func2(x_test, y_test))


Epoch 0 out of 50, loss: 138142.53160095215
Accuracy:  3904
Epoch 1 out of 50, loss: 111694.20513916016
Accuracy:  3842
Epoch 2 out of 50, loss: 90728.72596740723
Accuracy:  3900
Epoch 3 out of 50, loss: 74203.3357925415
Accuracy:  3915
Epoch 4 out of 50, loss: 60784.30824661255
Accuracy:  4023
Epoch 5 out of 50, loss: 50354.1953125
Accuracy:  4034
Epoch 6 out of 50, loss: 41507.71926116943
Accuracy:  3990
Epoch 7 out of 50, loss: 34064.859535217285
Accuracy:  4121
Epoch 8 out of 50, loss: 28092.03595352173
Accuracy:  4157
Epoch 9 out of 50, loss: 23219.942516326904
Accuracy:  4067
Epoch 10 out of 50, loss: 19132.151475906372
Accuracy:  4127
Epoch 11 out of 50, loss: 15955.313710212708
Accuracy:  4147
Epoch 12 out of 50, loss: 13075.859302520752
Accuracy:  4171
Epoch 13 out of 50, loss: 10714.423099040985
Accuracy:  4218
Epoch 14 out of 50, loss: 9259.935616016388
Accuracy:  4180
Epoch 15 out of 50, loss: 7473.301973581314
Accuracy:  4118
Epoch 16 out of 50, loss: 6520.881479978561
Acc

In [0]:
#print(func2(x_train,y_train))